In [1]:
import tkinter as tk
from tkinter import messagebox, ttk
from tkinter.simpledialog import askstring
import random

class BattleshipGUI:
    def __init__(self, master):
        self.master = master
        master.title("Battleships Game")

        self.game = Game()
        self.ai_player = AIPlayer("AI")
        self.ai_player.place_ships_automatically()
        self.player_boards = [[None for _ in range(10)] for _ in range(10)], [[None for _ in range(10)] for _ in range(10)]
        self.create_boards()
        self.status_label = tk.Label(master, text="Your Turn")
        self.status_label.grid(row=23, column=0, columnspan=10)
        player_name = askstring("Player Name", "Enter your name:")
        if not player_name:
            player_name = "Player 1"  # Default name if none entered
        self.player = Player(player_name)
        self.current_player_view = 0
        self.placement_mode = True
        self.ships_to_place = [
    ("Aircraft Carrier", 5),
    ("Battleship", 4),
    ("Cruiser", 3),
    ("Destroyer1", 2),  # First Destroyer
    ("Destroyer2", 2),  # Second Destroyer
    ("Submarine1", 1),  # First Submarine
    ("Submarine2", 1)   # Second Submarine
]# Add more ships as needed

        self.create_ship_placement_controls()
        #self.switch_board()  
        self.show_instructions_button = tk.Button(master, text="Show Instructions", command=self.show_instructions)
        self.show_instructions_button.grid(row=12, column=0, columnspan=10)
        self.show_instructions()


    def create_boards(self):
            # Create the labels for the columns
        for i in range(10):
            tk.Label(self.master, text=str(i+1)).grid(row=1, column=i)
            tk.Label(self.master, text=str(i+1)).grid(row=1, column=i+12)  # Offset for AI's board

        # Create the labels for the rows
        for i, letter in enumerate("ABCDEFGHIJ"):
            tk.Label(self.master, text=letter).grid(row=i+2, column=0)
            tk.Label(self.master, text=letter).grid(row=i+2, column=11)  # Offset for AI's board

        # Create the player's board
        for row in range(10):
            for col in range(10):
                button = tk.Button(self.master, text=' ', command=lambda r=row, c=col: self.on_button_click(r, c, 0))
                button.grid(row=row+2, column=col+1)  # +1 for the labels
                self.player_boards[0][row][col] = button

        # Create the AI's board
        for row in range(10):
            for col in range(10):
                button = tk.Button(self.master, text=' ', command=lambda r=row, c=col: self.on_button_click(r, c, 1))
                button.grid(row=row+2, column=col+13)  # +13 for the labels and the offset
                self.player_boards[1][row][col] = button

        # Player and AI labels
        player_label = tk.Label(self.master, text="Player's Board")
        player_label.grid(row=0, column=1, columnspan=10)

        ai_label = tk.Label(self.master, text="AI's Board")
        ai_label.grid(row=0, column=13, columnspan=10)
                    
         

    def create_ship_placement_controls(self):
        # Dropdown for ship selection
        self.ship_selection = ttk.Combobox(self.master, values=[ship[0] for ship in self.ships_to_place])
        
        self.ship_selection.current(0)

        # Radio buttons for orientation
        self.orientation_var = tk.StringVar(value="H")
        self.ship_selection.grid(row=22, column=0, columnspan=5)
        tk.Radiobutton(self.master, text="Horizontal", variable=self.orientation_var, value="H").grid(row=22, column=6, columnspan=2)
        tk.Radiobutton(self.master, text="Vertical", variable=self.orientation_var, value="V").grid(row=22, column=8, columnspan=2)

    def on_button_click(self, row, col, player):
        # Ensure that the player can only interact with their own board for placement
        # and only with the AI's board for attacks.
       
        if self.placement_mode and player == 0:  # Player's turn to place ships
            self.place_ship(row, col)
        elif not self.placement_mode and player == 1:  # Player's turn to attack AI's board
            self.make_attack(row, col)
            
            
    def place_ship(self, row, col):
        if not self.placement_mode:
            return  # Ignore if not in placement mode
        if self.current_player_view != 0:
            return  # Ignore if it's not the human player's turn

        ship_name = self.ship_selection.get()
        ship_index = [ship[0] for ship in self.ships_to_place].index(ship_name)
        ship_size = self.ships_to_place[ship_index][1]
        ship = Ship(ship_name, ship_size)

        # Use self.player to refer to the human player
        current_player = self.player

        orientation = self.orientation_var.get() == "H"

        if ship_name not in [ship[0] for ship in self.ships_to_place]:
            messagebox.showinfo("Invalid Action", "This ship has already been placed.")
            return
        
        if current_player.is_valid_placement(ship, row, col, orientation):
            current_player.board.add_ship(ship, row, col, orientation)
            for r, c in ship.coordinates:
                self.player_boards[0][r][c].config(text='S', bg='lightgrey')

            
            # Remove or disable the placed ship from the dropdown
            self.ships_to_place.pop(ship_index)
            self.ship_selection['values'] = [ship[0] for ship in self.ships_to_place]
            if self.ships_to_place:
                self.ship_selection.current(0)   
            else:
                self.placement_mode = False
                self.switch_player()  # Switch to the next player for ship placement or start the game
        else:
            messagebox.showinfo("Invalid Placement", "Can't place the ship here.")

            
    def make_attack(self, row, col):
        if self.current_player_view != 0:
            return

        result = self.player.make_attack(self.ai_player, row, col)

        # Update the AI's board view with the result
        self.update_ai_board_view()

        if "sunk" in result:
            messagebox.showinfo("You Sank My Ship!", result)
            # Clear AI's hit list if a ship is sunk
            self.ai_player.hits_to_follow_up.clear()

        if self.ai_player.board.all_ships_sunk():
            self.game_over(self.player.name)
        else:
            self.switch_player()

    def update_player_board_view(self):
        for row in range(10):
            for col in range(10):
                cell_content = self.player.board.grid[row][col]
                button = self.player_boards[0][row][col]
                if cell_content == "H":
                    button.config(text='H', bg='red')
                elif cell_content == "M":
                    button.config(text='M', bg='blue')
                # Ensure 'S' remains for player's ships
                elif cell_content == "S":
                    button.config(text='S', bg='lightgrey')
                else:
                    button.config(text=' ', bg='white')
                    
    def update_ai_board_view(self):
        for row in range(10):
            for col in range(10):
                cell_content = self.ai_player.board.grid[row][col]
                button = self.player_boards[1][row][col]
                if cell_content == "H":
                    button.config(text='H', bg='red')
                elif cell_content == "M":
                    button.config(text='M', bg='blue')
                else:
                    button.config(text=' ', bg='white')



    def update_board_view(self):
         # Update both the player's and the AI's boards.
        for row in range(10):
            for col in range(10):
                button = self.player_boards[0][row][col]
                cell_content = self.game.player.board.grid[row][col]
                if cell_content == "S":
                    button.config(text='S', bg='lightgrey')
                elif cell_content == "H":
                    button.config(text='H', bg='red')
                elif cell_content == "M":
                    button.config(text='M', bg='blue')
                else:
                    button.config(text=' ', bg='white')

        # Update the AI player's board.
        for row in range(10):
            for col in range(10):
                if self.game.ai_player.board.grid[row][col] == "S":
                    pass
                elif self.game.ai_player.board.grid[row][col] == "H":
                    self.player_boards[1][row][col].config(text='H', bg='red')
                elif self.game.ai_player.board.grid[row][col] == "M":
                    self.player_boards[1][row][col].config(text='M', bg='blue')
                else:
                    
                    pass

    def switch_player(self):
        if self.placement_mode:
            # If still placing ships, just switch the view
            self.current_player_view = 1 - self.current_player_view
        else:
            # Switch between player and AI turns
            if self.current_player_view == 0:  # If it's the player's turn
                self.current_player_view = 1
                self.status_label.config(text="AI's Turn")
                self.master.after(1000, self.ai_make_attack)  # AI attacks after a delay
            else:
                self.current_player_view = 0
                self.status_label.config(text=f"{self.player.name}'s Turn")

    def ai_place_ship(self):
        # Handle AI ship placement
        if not self.ai_player.all_ships_placed():
            self.ai_player.place_ships_automatically()
        else:
            self.placement_mode = False
            self.switch_player()
            
    def ai_make_attack(self):
            # Choose between smart attack and random attack
        if self.ai_player.hits_to_follow_up:
            row, col = self.ai_player.make_smart_attack()
        else:
            row, col = self.ai_player.make_random_attack()

        # Perform the attack
        result = self.ai_player.make_attack(self.player, row, col)

        # Update the player's board view with the result
        self.update_player_board_view()

        # Handle the result
        if "sunk" in result:
            messagebox.showinfo("AI Sank Your Ship!", result)

        if self.player.board.all_ships_sunk():
            self.game_over("AI")
        else:
            # Make sure the switch_player method is called here
            self.switch_player()

            
    def ai_turn(self):
        # AI's turn logic
        row, col = self.ai_player.make_random_attack()
        self.make_attack(row, col)
        
    def game_over(self, winner_name):
        result = messagebox.askquestion("Game Over", f"{winner_name} wins! Do you want to play again?")
        if result == 'yes':
            self.reset_game()
        else:
            self.master.quit()
            
    def show_instructions(self):
        instructions = """
        Battleship Game Instructions:
        - The goal is to sink all of your opponent's ships.
        - Each player takes turns to attack the opponent's board.
        - On your turn, choose a target on the opponent's board to attack.
        - If a ship occupies that square, it's a 'hit'; otherwise, it's a 'miss'.
        - The game ends when all ships of one player are sunk.
        - Ships are placed on the board vertically or horizontally.
        - Ships cannot overlap or be placed outside the board.
        """
        messagebox.showinfo("Game Instructions", instructions)
        
    
    def reset_game(self):
        # Reset the game state
        self.game = Game()
        self.player = Player(self.player.name)  # Retain the player's name
        self.ai_player = AIPlayer("AI")
        self.ai_player.place_ships_automatically()

        # Reset the boards
        for row in range(10):
            for col in range(10):
                self.player_boards[0][row][col].config(text=' ', bg='white')
                self.player_boards[1][row][col].config(text=' ', bg='white')

        # Reset ship placement
        self.placement_mode = True
        self.ships_to_place = [
            ("Aircraft Carrier", 5),
            ("Battleship", 4),
            ("Cruiser", 3),
            ("Destroyer1", 2),  # First Destroyer
            ("Destroyer2", 2),  # Second Destroyer
            ("Submarine1", 1),  # First Submarine
            ("Submarine2", 1)   # Second Submarine
        ]
        self.ship_selection['values'] = [ship[0] for ship in self.ships_to_place]
        self.ship_selection.current(0)
        self.ai_player.previous_attacks.clear()

        # Reset other states and UI elements as needed
        self.current_player_view = 0
        self.status_label.config(text=f"{self.player.name}'s Turn")




class Ship:
    def __init__(self, name, size):
        self.name = name
        self.size = size
        self.coordinates = []

    def place_ship(self, start_row, start_col, horizontal):
        self.coordinates = [(start_row + (i if not horizontal else 0), start_col + (i if horizontal else 0)) for i in range(self.size)]
class Board:
    def __init__(self):
        self.grid = [[" " for _ in range(10)] for _ in range(10)]
        self.ships = []

    def add_ship(self, ship, row, col, horizontal):
        ship.place_ship(row, col, horizontal)
        self.ships.append(ship)
        for r, c in ship.coordinates:
            self.grid[r][c] = "S"   

    def attack(self, row, col):
        if self.grid[row][col] == "S":
            self.grid[row][col] = "H"
            sunk_ship = self.check_if_ship_sunk(row, col)
            if sunk_ship:
                for r, c in sunk_ship.coordinates:
                    self.grid[r][c] = "H"  # Mark all parts of the ship as 'H'
                return f"Hit and sunk {sunk_ship.name}"
            return "Hit"
        elif self.grid[row][col] == " ":
            self.grid[row][col] = "M"
            return "Miss"
        else:
            return "Already Tried"
        
    def check_if_ship_sunk(self, row, col):
        for ship in self.ships:
            if (row, col) in ship.coordinates and all(self.grid[r][c] == "H" for r, c in ship.coordinates):
                return ship
        return None
    
    def all_ships_sunk(self):
        return all(self.grid[r][c] == "H" for ship in self.ships for r, c in ship.coordinates)
    
class Player:
    def __init__(self, name):
        self.name = name
        self.board = Board()

    def is_valid_placement(self, ship, row, col, horizontal):
        if horizontal:
            if col + ship.size > 10:  # Check horizontal bounds
                return False
        else:
            if row + ship.size > 10:  # Check vertical bounds
                return False

        for i in range(ship.size):
            r = row + (i if not horizontal else 0)
            c = col + (i if horizontal else 0)
            if self.board.grid[r][c] != " ":
                return False  # Ship overlaps or is out of bounds

        return True
    
    def make_attack(self, opponent, row, col):
        
        return opponent.board.attack(row, col)

class Game:
    def __init__(self):
        self.player = Player("Player 1")
        self.ai_player = AIPlayer("AI")

        
class AIPlayer(Player):
    def __init__(self, name):
        super().__init__(name)
        self.previous_attacks = set()
        self.hits_to_follow_up = []
    
    def place_ships_automatically(self):
        for ship_name, ship_size in [
    ("Aircraft Carrier", 5),
    ("Battleship", 4),
    ("Cruiser", 3),
    ("Destroyer1", 2),  # First Destroyer
    ("Destroyer2", 2),  # Second Destroyer
    ("Submarine1", 1),  # First Submarine
    ("Submarine2", 1)   # Second Submarine
]:
            placed = False
            while not placed:
                row = random.randint(0, 9)
                col = random.randint(0, 9)
                horizontal = random.choice([True, False])

                ship = Ship(ship_name, ship_size)
                if self.is_valid_placement(ship, row, col, horizontal):
                    self.board.add_ship(ship, row, col, horizontal)
                    placed = True
                else:
                    print(f"Failed placement: {ship_name} at {(row, col)} horizontal: {horizontal}")

    def all_ships_placed(self):
        return len(self.board.ships) == len([
    ("Aircraft Carrier", 5),
    ("Battleship", 4),
    ("Cruiser", 3),
    ("Destroyer1", 2),  # First Destroyer
    ("Destroyer2", 2),  # Second Destroyer
    ("Submarine1", 1),  # First Submarine
    ("Submarine2", 1)   # Second Submarine
])  # Check if all ships are placed

    def make_smart_attack(self):
        if self.hits_to_follow_up:
            last_hit = self.hits_to_follow_up[-1]
            # Generate potential target cells around the last hit (up, down, left, right)
            potential_targets = [
                (last_hit[0] - 1, last_hit[1]), 
                (last_hit[0] + 1, last_hit[1]), 
                (last_hit[0], last_hit[1] - 1), 
                (last_hit[0], last_hit[1] + 1)
            ]
            # Filter out invalid or already attacked cells
            valid_targets = [
                (row, col) for row, col in potential_targets 
                if 0 <= row < 10 and 0 <= col < 10 and (row, col) not in self.previous_attacks
            ]
            if valid_targets:
                return random.choice(valid_targets)
        return self.make_random_attack()

    def make_random_attack(self):
        row, col = random.randint(0, 9), random.randint(0, 9)
        while (row, col) in self.previous_attacks:
            row, col = random.randint(0, 9), random.randint(0, 9)
        self.previous_attacks.add((row, col))
        return row, col

    def make_attack(self, opponent, row, col):
        result = super().make_attack(opponent, row, col)
        if "Hit" in result:
            self.hits_to_follow_up.append((row, col))
        elif "sunk" in result:
            self.hits_to_follow_up.clear()
        return result
        
if __name__ == "__main__":
    root = tk.Tk()
    root.minsize(800, 400)  
    gui = BattleshipGUI(root)
    root.mainloop()


Failed placement: Battleship at (8, 2) horizontal: False
Failed placement: Destroyer2 at (2, 5) horizontal: True
